# <p style="text-align: center;">COMP 4304 – Second Project</p>

## Team Members:
- Wasif Ibrar (202149944)
- Muhammad Mahad Mirza (202141537)

## Required imports

In [2]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

## Read the Dataset

In [3]:
data =  pd.read_csv("open_gym.csv")

## Data Manipulation and Pre-processing

In [4]:
data['open_gym_start'] = pd.to_datetime(data['open_gym_start'])
data['open_gym_end'] = pd.to_datetime(data['open_gym_end'])

## Dashboard

In [9]:
# Define color scales and palettes
color_scale_activity = px.colors.qualitative.Pastel
color_scale_gender = px.colors.qualitative.Light24

# Initialize the Dash app with an external stylesheet for aesthetics
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Generate dropdown options for filtering activities and gender
activity_options = [{'label': 'All Activities', 'value': 'All'}] + \
                   [{'label': activity, 'value': activity} for activity in data['open_gym_activity'].unique()]
gender_options = [{'label': 'All', 'value': 'Both'},
                  {'label': 'Male', 'value': 'total_males'},
                  {'label': 'Female', 'value': 'total_females'}]

# Define the app layout
app.layout = html.Div([
    html.H1("City's Parks and Recreation Dashboard", style={'textAlign': 'center'}),
    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='Activity Over Time', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=data['open_gym_start'].min().date(),
                    end_date=data['open_gym_start'].max().date(),
                    min_date_allowed=data['open_gym_start'].min().date(),
                    max_date_allowed=data['open_gym_start'].max().date()),
                dcc.Dropdown(
                    id='activity-dropdown',
                    options=activity_options,
                    value='All',
                    clearable=False),
                dcc.Graph(id='activity-over-time-graph')
            ], style={'padding': '20px'})
        ]),
        dcc.Tab(label='Community Center Comparison', children=[
            html.Div(dcc.Graph(id='community-center-comparison-graph'), style={'padding': '20px'})
        ]),
        dcc.Tab(label='Popularity Comparison', children=[
            html.Div(dcc.Graph(id='popularity-comparison-graph'), style={'padding': '20px'})
        ]),
        dcc.Tab(label='Gender Distribution', children=[
            html.Div([dcc.Dropdown(
                    id='gender-dropdown',
                    options=gender_options,
                    value='Both',
                    clearable=False),
                     
                dcc.Graph(id='gender-distribution-graph')], style={'padding': '20px'})
        ])
    ])
])

# Callback for the Activity Over Time graph
@app.callback(
    Output('activity-over-time-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('activity-dropdown', 'value')]
)
def update_activity_over_time(start_date, end_date, selected_activity):
    filtered_data = data[(data['open_gym_start'] >= start_date) & (data['open_gym_end'] <= end_date)]
    
    if selected_activity != 'All':
        # Filter for the selected activity
        filtered_data = filtered_data[filtered_data['open_gym_activity'] == selected_activity]
        # Here, we don't group by activity since we're only focusing on one activity.
        # Instead, we group by the start date to aggregate counts per month.
        filtered_data = filtered_data.groupby(pd.Grouper(key='open_gym_start', freq='M')).size().reset_index(name='total')
        # Since there's only one activity, we manually set the activity column to reflect the selected activity for consistent plotting.
        filtered_data['open_gym_activity'] = selected_activity
    else:
        # For "All Activities", group by both month and activity
        filtered_data = filtered_data.groupby([pd.Grouper(key='open_gym_start', freq='M'), 'open_gym_activity']) \
                                     .size().reset_index(name='total')

    title = f'Activity Over Time: {selected_activity if selected_activity != "All" else "All Activities"}'
    fig = px.line(filtered_data, x='open_gym_start', y='total', color='open_gym_activity', title=title, color_discrete_sequence=color_scale_activity)
    fig.update_layout(xaxis_title='Year', yaxis_title='Total Participants', xaxis=dict(tickformat='%Y'))
    return fig

# Callback for Community Center Comparison
@app.callback(
    Output('community-center-comparison-graph', 'figure'),
    Input('tabs', 'value')
)
def update_community_center_comparison(_):
    grouped_data = data.groupby(['facility_title', 'open_gym_activity']).size().reset_index(name='counts')
    fig = px.bar(grouped_data, x='facility_title', y='counts', color='open_gym_activity', title='Community Center Comparison', barmode='stack', color_discrete_sequence=color_scale_activity)
    return fig

# Callback for Popularity Comparison
@app.callback(
    Output('popularity-comparison-graph', 'figure'),
    Input('tabs', 'value')
)
def update_popularity_comparison(_):
    grouped_data = data.groupby('open_gym_activity')['total'].sum().reset_index()
    fig = px.pie(grouped_data, values='total', names='open_gym_activity', title='Popularity Comparison', color_discrete_sequence=color_scale_activity)
    return fig

# Callback for Gender Distribution
@app.callback(
    Output('gender-distribution-graph', 'figure'),
    Input('gender-dropdown', 'value'),
)
def update_gender_distribution(selected_gender):
    if selected_gender == 'Both':
        gender_data = data.groupby('open_gym_activity')[['total_females', 'total_males']].sum().reset_index()
        fig = go.Figure()
        for gender, color in zip(['total_females', 'total_males'], color_scale_gender):
            fig.add_trace(go.Scatterpolar(
                r=gender_data[gender],
                theta=gender_data['open_gym_activity'],
                fill='toself',
                name=gender.split('_')[1].capitalize(),
                line=dict(color=color)
            ))
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True)),
            title='Gender Distribution by Activity'
        )
    elif selected_gender in ['total_males', 'total_females']:
        gender_data = data.groupby('open_gym_activity')[selected_gender].sum().reset_index()
        fig = go.Figure()
        fig.add_trace(go.Scatterpolar(
            r=gender_data[selected_gender],
            theta=gender_data['open_gym_activity'],
            fill='toself',
            name=selected_gender.split('_')[1].capitalize(),
            line=dict(color=color_scale_gender[1] if selected_gender == 'total_females' else color_scale_gender[0])
        ))
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True)),
            title=f'Gender Distribution by Activity - {selected_gender.split("_")[1].capitalize()}'
        )
    else:
        fig = go.Figure()
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True)),
            title='Gender Distribution by Activity'
        )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8003, use_reloader=False)


# Attributions

- Dash Documentation: https://dash.plotly.com/
- Colorscheme: https://stackoverflow.com/questions/60962274/plotly-how-to-change-the-colorscheme-of-a-plotly-express-scatterplot
- Date Picker:
    1. https://dash.plotly.com/dash-core-components/datepickerrange
    2. https://stackoverflow.com/questions/69017021/dash-datepickerrange-with-graph
- Drop Down: https://dash.plotly.com/dash-core-components/dropdown
- CSS
    1. https://dash.plotly.com/external-resources
    2. https://codepen.io/chriddyp/pen/bWLwgP
- Grouper for activity graph: https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html
- scatterplot: https://plotly.com/python/reference/scatterpolar/
- Chat GPT for making the code more efficient and ideas for visualisations.